# Minimal examples for join, merge & concat 

1) Index-on-index join with defaults <br>
By default, every DataFrame has a RangeIndex starting at 0

In [44]:
import pandas as pd
df1 = pd.DataFrame({"A": [10, 20]}) # index 0,1
df2 = pd.DataFrame({"B": ["x", "y"]}) # index 0,1
df1.join(df2) # works, aligns 0 with 0, 1 with 1

,A,B
0,10,x
1,20,y


2) join with overlapping columns (requires suffixes)

In [45]:
df1 = pd.DataFrame({"A": [1, 2]})
df2 = pd.DataFrame({"A": [100, 200]})
df1.join(df2, lsuffix="_left", rsuffix="_right")

,A_left,A_right
0,1,100
1,2,200


3) merge with overlapping non-key columns (auto suffixes)

In [47]:
df1 = pd.DataFrame({"key": [1, 2], "A": [1, 2]})
df2 = pd.DataFrame({"key": [1, 2], "A": [100, 200]})
pd.merge(df1, df2, on="key") # -> A_x, A_y (automatically done by merge)

,key,A_x,A_y
0,1,1,100
1,2,2,200


4. concat(axis=1) just glues columns together by index. If indexes don’t match, you’ll see NaNs.

In [48]:
df1 = pd.DataFrame({"A": [10, 20]})
df2 = pd.DataFrame({"B": ["x", "y"]})
cc = pd.concat([df1, df2], axis=1)
print(cc)

    A  B
0  10  x
1  20  y


In [43]:
df1 = pd.DataFrame({"A": [10, 20]})
df2 = pd.DataFrame({"B": ["x", "y"]})
cc = pd.concat([df1, df2], axis=0)
print(cc)

      A    B
0  10.0  NaN
1  20.0  NaN
0   NaN    x
1   NaN    y


# Examples with Simple Common Dataset for All operations/methods 

In [58]:
import pandas as pd

# Employees
employees = pd.DataFrame({
    "EmpID": [1, 2, 3],
    "Name": ["Alice", "Bob", "Charlie"]
})

# Departments
departments = pd.DataFrame({
    "EmpID": [1, 2],
    "Dept": ["HR", "IT"]
})

# Salaries
salaries = pd.DataFrame({
    "EmpID": [2, 3],
    "Salary": [50000, 60000]
})

print(employees,end="\n\n")
print(departments,end="\n\n")
print(salaries)

   EmpID     Name
0      1    Alice
1      2      Bob
2      3  Charlie

   EmpID Dept
0      1   HR
1      2   IT

   EmpID  Salary
0      2   50000
1      3   60000


Notice in the Dataset (data frames above):
- 'Charlie' has no department.
- 'Alice' has no salary.
- This gives us realistic mismatches for few usecases.

### Join Examples

##### A. Index-on-index join with defaults

In [59]:
# Set EmpID as index for both DataFrames
employee_DataFrame_Indexed = employees.set_index("EmpID")
department_DataFrame_Indexed = departments.set_index("EmpID")

joined = employee_DataFrame_Indexed.join(department_DataFrame_Indexed)  # index-on-index
print(joined)


          Name Dept
EmpID              
1        Alice   HR
2          Bob   IT
3      Charlie  NaN


Note that 'Charlie' won't have any department in the output/result above.<br>
Index alignment: Charlie has no department → NaN.

##### B. Join with overlapping columns (requires suffixes)

In [60]:
# Both employee dataFrame (created earlier above) and oldEmployeeRecord (2024 Employee Snapshot) have 'Name' column
oldEmployeeRecord_Indexed = pd.DataFrame({
    "EmpID": [1, 2],
    "Name": ["Alicia", "Bob R."] # slightly different spelling, say, maybe by sales department in year 2024. 
}).set_index("EmpID")

employee_DataFrame_Indexed.join(oldEmployeeRecord_Indexed, lsuffix="_fromRecentYearTable", rsuffix="_fromYear2024Table") # here we use word 'Table' instead of 'DataFrame' for simplicity


,Name_fromRecentYearTable,Name_fromYear2024Table
EmpID,,
1,Alice,Alicia
2,Bob,Bob R.
3,Charlie,NaN


Without suffixes, Pandas would error. With suffixes, both columns are kept.

**Notes**: Real-life cases where overlapping column names occur:
- Different systems storing the same attribute (Name, Dept).
- Different versions of data (yearly snapshots).
- Different formatting or spelling of the same field.
<p>That’s why Pandas allows overlapping names but forces us to suffix them — so we don’t silently overwrite one source of truth. 

### Merge Example (SQL-style)

##### Column-to-column join. Alice has no salary → NaN.

In [62]:
merged = pd.merge(employees, salaries, on="EmpID", how="left")
print(merged)

   EmpID     Name   Salary
0      1    Alice      NaN
1      2      Bob  50000.0
2      3  Charlie  60000.0


### Concat Example

Concat just glued them together.

In [63]:
# Concatenate employees and departments side by side
cc = pd.concat([employees, departments], axis=1)
print(cc)


   EmpID     Name  EmpID Dept
0      1    Alice    1.0   HR
1      2      Bob    2.0   IT
2      3  Charlie    NaN  NaN


> Notes: What does axis=0 vs axis=1 mean in concat?

***axis=0 → rows (vertical direction)***  therefore, ***concat(axis=0) → stack rows*** 
<br>(you can reset/ignore duplicate row labels/indexes in the output/result with ignore_index=True )

***axis=1 → columns (horizontal direction)*** therefore, ***concat(axis=1) → add columns side by side***






Notice the result after you execute the above code that duplicate EmpID columns, and Charlie’s row has NaNs because departments only had 2 rows.

### Summary

| Method | How it Works | Example Result |
|:--------:|:--------:|:--------:|
|  join   |  Index vs index (or column vs index)   |  Employees + Departments by EmpID index   |
|  merge   |  Column vs column (SQL-like)   |  Employees + Salaries by EmpID   |
|  concat   |  Glue side by side or top/bottom   |  Employees + Departments stacked, duplicate EmpID   |

### Takeaway

- Use merge when you want SQL-style joins on columns.

- Use join when your DataFrames are "***already indexed by the key***".

- Use concat when you just want to glue DataFrames together (rows or columns), without matching keys.

<br>
Additional Note:

“***already indexed by the key***” means we have set a meaningful column (like CustomerID, Date) as the DataFrame’s index.

By default, Pandas gives a RangeIndex (0,1,…), so we usually set the index ourself after importing.

***join*** is most convenient when both DataFrames are ***already indexed by the same key***.

If not, ***merge*** is the safer, SQL‑like choice.
